## Interfaces

An `interface` is a defnition of an API. It describes a list of methods and attributes a
class should have to implement with the desired behavior. This description does not
implement any code but just defines an explicit contract for any class that wishes to
implement the interface. Any class can then implement one or several interfaces in
whichever way it wants.

### Using `zope.interface`

The core class of the `zope.interface` package is the `Interface` class. It allows you
to explicitly defne a new interface by subclassing. Let's assume that we want to
defne the obligatory interface for every implementation of a rectangle:

In [18]:
from zope.interface import Interface, Attribute, implementer
from zope.interface.verify import verifyClass, verifyObject

class IRectangle(Interface):
    width = Attribute("The width of rectangle")
    height = Attribute("The height of rectangle")
    
    def area():
        """Return area of rectangle"""
        pass
    
    def perimeter():
        """Return perimeter of rectangle"""
        pass

• The common naming convention for interfaces is to use `I` as the name suffix.  
• The methods of the interface must not take the `self` parameter.  
• As the interface does not provide concrete implementation, it should
consist only of empty methods. You can use the `pass` statement, raise
`NotImplementedError`, or provide a `docstring` (preferred).  
• An interface can also specify the required attributes using the `Attribute` class.

In [19]:
@implementer(IRectangle)
class Square:
    """Concrete implementation of square with rectangle interface"""
    def __init__(self, size):
        self.size = size
    
    @property
    def width(self):
        return self.size
    
    @property
    def height(self):
        return self.size
    
    def area(self):
        return self.size ** 2
    
    def perimeter(self):
        return 4 * self.size

In [ ]:
@implementer(IRectangle)
class Square:
    """Concrete implementation of square with rectangle interface"""
    def __init__(self, size):
        self.size = size
    
    @property
    def width(self):
        return self.size
    
    @property
    def height(self):
        return self.size
    
    def area(self):
        return self.size ** 2
    
    def perimeter(self):
        return 4 * self.size

In [23]:
a = Square(5)
print(verifyClass(IRectangle, Square))
print(verifyObject(IRectangle, a))

True
True


In [25]:
# Not implement some properties
@implementer(IRectangle)
class Circle:
    def __init__(self, radius):
        self.radius = radius
    def area(self):
        return math.pi * self.radius ** 2
    def perimeter(self):
        return 2 * math.pi * self.radius

b = Circle(3)
print(verifyClass(IRectangle, Circle))  # True
print(verifyObject(IRectangle, b))  # Not implementation

True


BrokenImplementation: An object has failed to implement interface <InterfaceClass __main__.IRectangle>

        The width attribute was not provided.
        

### Using Abstract classes(ABC)

In [26]:
from abc import ABCMeta, abstractmethod

In [28]:
class Pushable(metaclass=ABCMeta):
    @abstractmethod
    def push(self, x):
        """ Push argument no matter what it means """    

In [30]:
class DummyPushable(Pushable):
    def push(self, x):
        return
    
class IncompletePushable(Pushable):
    pass

In [31]:
DummyPushable()

In [33]:
IncompletePushable()  # raise error

TypeError: Can't instantiate abstract class IncompletePushable with abstract methods push

a great way to ensure implementation completeness of base classes but is as explicit as the `zope.interface` alternative

In [35]:
# not a subclass of `Pushable`, so although implement the base method, it is not identified.
# but in zope.interface, it will be identified.
class SomethingWithPush:
    def push(self, x):
        pass

isinstance(SomethingWithPush, Pushable)

False

In [37]:
from abc import ABCMeta, abstractmethod

class Pushable(metaclass=ABCMeta):
    
    @abstractmethod
    def push(self, x):
        """ Push argument no matter what it means """
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Pushable:
            if any("push" in sub.__dict__ for sub in C.__mro__):
                return True
        return NotImplemented

In [38]:
isinstance(SomethingWithPush(), Pushable)

True

### Using function annotations to implement

In [42]:
from abc import ABCMeta, abstractmethod, abstractproperty

# a method to simulate zope.interface using abc
class IRectangle(metaclass=ABCMeta):
    
    @abstractproperty
    def width(self):
        pass
    
    @abstractproperty
    def height(self):
        pass
    
    @abstractmethod
    def area():
        """Return area of rectangle"""
        pass
    
    @abstractmethod
    def perimeter():
        """Return perimeter of rectangle"""
        pass
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is IRectangle:
            if all([
                any("area" in sub.__dict__ for sub in C.__mro__),
                any("perimeter" in sub.__dict__ for sub in C.__mro__),
                any("width" in sub.__dict__ for sub in C.__mro__),
                any("height" in sub.__dict__ for sub in C.__mro__),
            ]):
                return True
        return NotImplemented

In [44]:
import inspect
from functools import wraps

In [45]:
def ensure_interface(function):
    signature = inspect.signature(function)
    parameters = signature.parameters
    
    @wraps(function)
    def wrapped(*args, **kwargs):
        bound = signature.bind(*args, **kwargs)
        for name, value in bound.arguments.items():
            annotation = parameters[name].annotation
            if not isinstance(annotation, ABCMeta):
                continue
            if not isinstance(value, annotation):
                raise TypeError("{} does not implement {} interface".format(value, annotation))
        return function(*args, **kwargs)
    return wrapped

In [46]:
class ImplicitRectangle:
    
    def __init__(self, width, height):
        self._width = width
        self._height = height

    @property
    def width(self):
        return self._width
    
    @property
    def height(self):
        return self._height
    
    def area(self):
        return self.width * self.height
    
    def perimeter(self):
        return self.width * 2 + self.height * 2

@ensure_interface
def draw_rectangle(rectangle: IRectangle):
    print("{} x {} rectangle drawing.".format(rectangle.width, rectangle.height))

In [53]:
draw_rectangle("foo")

TypeError: foo does not implement <class '__main__.IRectangle'> interface

In [54]:
draw_rectangle(ImplicitRectangle(2, 10))

2 x 10 rectangle drawing.


Another choice is to use `collections.abc` which already provide a lot of predefned ABCs that allow to verify interface compatibility of many basic Python types. 